# Dimensionality reduction

The feature selection and extraction process often leaves us with a very large featureset: each document may be characterized by thousands of token frequencies, for example. In order to effectively look for patterns within these data and visualize the results, we need meaningful ways to prune or combine features.

If this is a new R session, we’ll need to re-load the tidyverse packages.

In [1]:
library(tidyverse)
library(tidytext)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Topic Modeling

Topic modeling is the process of identifying constellations of tokens that tend to co-occur among the documents. This can have two benefits. First, the number of such token groups, or **topics**, is much smaller than the number of tokens. Characterizing the documents in terms of, say, 20 topics is simpler than trying to represent them in terms of thousands of word frequencies. The second potential benefit is that the topics themselves can often be treated as meaningful. Understanding which words contribute most to each topic may grant some insight into the connections between ideas within the documents.

A number of methods can be used for topic modeling, finding the one that works best for your dataset generally involves some experimentation. It is almost always necessary to tune the model based on some test data in order to get the best performance. Today we’ll try some simple experiments with **Latent Dirichlet Allocation** LDA, a technique popularized in the 2000s by David Blei. Specifically, we’ll use the version implemented by the `topicmodels` package.

The results are not guaranteed to be identical every time you run the algorithm, even when you use the same parameters. The model is probabilistic, and the computer builds it by an iterative trial and error process that contains some randomness. You can explicitly set a random number "seed" if you want to make sure that the same set of pseudo-random numbers is used every time. But it’s also helpful to do several runs with different seeds and indeed with slightly different starting parameters and confirm that the results are comparable.

## Choosing a feature set and number of topics

LDA can take a long time. If you are able to fine-tune your feature set to remove very common and very rare terms first, the process will be more efficient and accurate. Unfortunately, determining the number of very common and very rare terms to prune is often itself a trial and error process.

Likewise for ***k***, the number of topics. It is common to re-run the process with many different values of k in order to optimize for the most stable results. Some packages even provide a script to help automate this process.

## Additional datasets

For this example, we can use the **sotu** package, which provides the texts of the annual "State of the Union" address by the President of the United States. I’m also going to use the pre-made English stoplist from the package **NLP** and the English stemmer from **SnowballC**.

In [2]:
library(topicmodels)
library(NLP)
library(SnowballC)
library(sotu)


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




The **sotu** package provides the texts of the "State of the Union" addresses as a character vector called `sotu_text`. It also provides additional information about each speech, including the year, and the name and party of the President, in a tibble called `sotu_meta`. I’m going to add an `id` column to both so I can correlate them later.

In [3]:
head(sotu_meta)

president,year,years_active,party,sotu_type
<chr>,<int>,<chr>,<chr>,<chr>
George Washington,1790,1789-1793,Nonpartisan,speech
George Washington,1790,1789-1793,Nonpartisan,speech
George Washington,1791,1789-1793,Nonpartisan,speech
George Washington,1792,1789-1793,Nonpartisan,speech
George Washington,1793,1793-1797,Nonpartisan,speech
George Washington,1794,1793-1797,Nonpartisan,speech


In [4]:
sotu_meta <- sotu_meta %>% 
    mutate(id = row_number()) %>%
    relocate(id, .before = president)

In [5]:
head(sotu_meta)

id,president,year,years_active,party,sotu_type
<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,George Washington,1790,1789-1793,Nonpartisan,speech
2,George Washington,1790,1789-1793,Nonpartisan,speech
3,George Washington,1791,1789-1793,Nonpartisan,speech
4,George Washington,1792,1789-1793,Nonpartisan,speech
5,George Washington,1793,1793-1797,Nonpartisan,speech
6,George Washington,1794,1793-1797,Nonpartisan,speech


In [6]:
str(sotu_text)

 chr [1:236] "Fellow-Citizens of the Senate and House of Representatives: \n\nI embrace with great satisfaction the opportuni"| __truncated__ ...


In [7]:
sotu_text <- tibble(text=sotu_text) %>%
    mutate(id = row_number()) %>%
    relocate(id, .before=text)

In [8]:
print(sotu_text)

# A tibble: 236 × 2
      id text                                                                   
   <int> <chr>                                                                  
 1     1 "Fellow-Citizens of the Senate and House of Representatives: \n\nI emb…
 2     2 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 3     3 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 4     4 "Fellow-Citizens of the Senate and House of Representatives: \n\nIt is…
 5     5 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 6     6 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 7     7 "\n\nFellow-Citizens of the Senate and House of Representatives: \n\nI…
 8     8 "\n\n Fellow-Citizens of the Senate and House of Representatives: \n\n…
 9     9 "\n\n Gentlemen of the Senate and Gentlemen of the House of Representa…
10    10 "\n\n Gentlemen of the Senate and Gentlemen of the House of Representa…
# … with

### Tokenization

The first step is feature extraction. Let’s begin by tokenzing into words and stemming. I'm using SnowballC’s `wordStem()` for stemming. This is a quick-and-dirty method for getting at the underlying "stems" of words that for the most part just chops off anything that looks like an inflectional ending, with no sensitivity to context.

In [9]:
tokens <- sotu_text %>% 
    unnest_tokens(word, text) %>%
    mutate(stem = wordStem(word))

In [10]:
tokens %>% head(20)

id,word,stem
<int>,<chr>,<chr>
1,fellow,fellow
1,citizens,citizen
1,of,of
1,the,the
1,senate,senat
1,and,and
1,house,hous
1,of,of
1,representatives,repres


Let’s calculate corpus-wide stem counts. Then we can make a list of overly-common stems to be omitted from the topic model.

In [11]:
corpus_stem_counts <- tokens %>% count(stem, sort=TRUE)

In [12]:
corpus_stem_counts %>% head(25)

stem,n
<chr>,<int>
the,164556
of,105824
to,67358
and,67193
a,44263
in,43013
i,29211
that,23845
it,22386


Just for now, I’m going to make a MFW list of everything up to 'state' and exclude those terms. In a more careful study I would want to hand-select stopwords here.

In [13]:
stop_mfw <- corpus_stem_counts$stem[1:24]
print(stop_mfw)

 [1] "the"   "of"    "to"    "and"   "a"     "in"    "i"     "that"  "it"   
[10] "be"    "for"   "our"   "by"    "have"  "on"    "thi"   "which" "we"   
[19] "with"  "will"  "ha"    "ar"    "been"  "not"  


To generate a list of *infrequent* words we want to exclude, let’s calculate per-document word counts, and then determine how many documents each word occurs in.

In [14]:
stem_doc_counts <- tokens %>% 
    count(id, stem) %>%
    count(stem, sort=TRUE)

In [15]:
stem_doc_counts

stem,n
<chr>,<int>
a,236
all,236
an,236
and,236
ar,236
at,236
be,236
been,236
but,236


Let’s make a list of words that occur in five documents or fewer.

In [16]:
stop_lfw <- with(stem_doc_counts, stem[n<=4])

In [17]:
length(stop_lfw)

[1] 12823

In [18]:
head(stop_lfw, 100)

[1] "1,200"       "1.5m"        "103"         "107"         "110,000"    
  [6] "11m"         "125"         "140"         "140,000,000" "1753"       
 [11] "180"         "180,000"     "1806"        "1807"        "1843"       
 [16] "18m"         "19,000,000"  "1923"        "1952"        "1956"       
 [21] "196"         "1965"        "1966"        "1967"        "1986"       
 [26] "1994"        "1997"        "1998"        "2.2"         "2.4"        
 [31] "2.6"         "2007"        "2014"        "22,000"      "271"        
 [36] "290"         "3,100"       "3,500,000"   "3.2"         "3.3"        
 [41] "3.65"        "32,000,000"  "33,000,000"  "346,681,016" "38,000"     
 [46] "400,000,000" "425,000"     "450"         "46"          "47,000,000" 
 [51] "4m"          "5,500,000"   "504"         "54,000,000"  "60,000,000" 
 [56] "64"          "650"         "675"         "700,000,000" "71"         
 [61] "77"          "8.5"         "8.5m"        "80th"        "88"         
 [66] "93"          "95th"        "96"          "abhor"       "abomin"     
 [71] "acid"        "activist"    "adhes"       "admiralti"   "affront"    
 [76] "aforesaid"   "aimless"     "alfr"        "aluminum"    "amen"       
 [81] "american'"   "amphitrit"   "angl"        "anguish"     "anthracit"  
 [86] "antonio"     "antwerp"     "apollo"      "appendix"    "arabian"    
 [91] "aristocraci" "aristocrat"  "army'"       "asean"       "astronaut"  
 [96] "astronomi"   "asund"       "attaind"     "auction"     "auguri"

Let’s apply these stoplists to our token table and generate a new set of counts.

In [19]:
token_counts_no_stops <- tokens %>% 
    filter(! stem %in% stop_mfw) %>%
    filter(! stem %in% stop_lfw) %>%
    count(id, stem, sort=TRUE)

In [20]:
print(token_counts_no_stops)

# A tibble: 308,810 × 3
      id stem         n
   <int> <chr>    <int>
 1   199 year       247
 2   158 year       227
 3   119 should     212
 4   158 dollar     212
 5   199 my         205
 6   199 congress   204
 7   158 war        202
 8   199 program    200
 9   201 year       192
10   122 govern     185
# … with 308,800 more rows


### Convert to document-term matrix

In order to use `topicmodels`, we need to convert this tibble, which has one row for each document-term combination, to an enormous matrix that has one row for every document and one column for every stem. Note that most of the values in this matrix will be zero, since most of the stems don’t occur in most of the documents. That mans that this is a "sparse matrix", and R has special ways of dealing with these.

In particular, a number of language-processing packages share a common DTM structure, and tidytext knows how to convert our data to this format. The conversion function is called `cast_dtm()`

In [21]:
dtm <- token_counts_no_stops %>% cast_dtm(id, stem, n)

What are the dimensions of the resulting matrix?

In [22]:
dim(dtm)

[1]  236 6950

So we have 236 documents, for which we’ve measured 6950 stem counts. Using `print()` on the DTM will tell us a little more about its properties, if we’re interested.

In [23]:
print(dtm)

<<DocumentTermMatrix (documents: 236, terms: 6950)>>
Non-/sparse entries: 308810/1331390
Sparsity           : 81%
Maximal term length: 15
Weighting          : term frequency (tf)


## Building a model

Now we can go ahead and run the LDA algorithm to build a topic model. In the interest of time, I’m only going to ask it to generate five topics. That’s probably too few, but with luck it will be illustrative. On my computer this step takes about a minute or two.

In [24]:
lda_model <- LDA(dtm, k=5)

The resulting model is a complex R object, with lots of component attributes. In particular, we’re interested in "beta" the probabilities that each stem is associated with each topic, and "gamma", the probabilities that each topic is associated with each document.

Each of these is a matrix:
 - beta has one row for each topic and a column for every term
     - each topic is seen as made up of all the terms in various proportions
 - gamma has one row for each document and a column for every topic
     - each document is seen as made up of all the topics in various proportions
     
One way to get these values out of the model is with the helper function `posterior()`.

In [25]:
posterior(lda_model)

$terms
         year      should       dollar           my    congress          war
1 0.003793541 0.003293072 7.166853e-05 0.0026463429 0.005281880 0.0034162093
2 0.007826724 0.006129544 7.137557e-04 0.0017516645 0.005462962 0.0006452264
3 0.000787897 0.010492623 9.158176e-05 0.0005242223 0.002261983 0.0064745194
4 0.009357495 0.002916561 6.221487e-04 0.0027709465 0.003753715 0.0013039554
5 0.011326092 0.001583986 2.231428e-03 0.0044083768 0.007124866 0.0018847413
       program      govern          wa       state      nation       mexico
1 1.803868e-37 0.008977204 0.007782795 0.013610248 0.004760946 2.096239e-03
2 4.035473e-22 0.010208838 0.006908082 0.008397109 0.001554149 1.201665e-04
3 1.184268e-10 0.005132065 0.002233756 0.003226343 0.011393789 1.679010e-04
4 1.703761e-03 0.003218004 0.002759754 0.002948157 0.004776359 4.600941e-05
5 9.096761e-03 0.006349124 0.002210003 0.005470465 0.008043100 8.739427e-05
           or       their        feder       legisl      million       propos
1 0.005529990 0.010150171 0.0005129969 0.0008673728 0.0003579143 0.0008316755
2 0.005576657 0.004868786 0.0006581351 0.0028701457 0.0003032455 0.0006310014
3 0.010060216 0.007878735 0.0006436415 0.0011297732 0.0005179471 0.0005775892
4 0.004595395 0.007119552 0.0012773227 0.0004158428 0.0022126435 0.0016043210
5 0.001736711 0.003801087 0.0070830627 0.0041803262 0.0031737060 0.0042343425
     administr      develop      continu         such        from         all
1 4.509072e-04 9.710474e-05 0.0016314578 0.0042476723 0.009903489 0.006744483
2 1.506733e-03 8.745900e-04 0.0014812543 0.0060033517 0.006337959 0.004210180
3 3.026971e-05 1.501328e-03 0.0007904558 0.0045806464 0.005800655 0.009449739
4 4.137744e-04 6.585902e-04 0.0013736587 0.0002596752 0.005577554 0.007966241
5 4.425126e-03 5.483717e-03 0.0055398726 0.0016433157 0.004047391 0.003631551
        fiscal   expenditur         work       polici          new        unit
1 5.342045e-04 1.096679e-03 0.0003903506 0.0014402654 0.0018667912 0.006923901
2 1.675892e-03 1.625177e-03 0.0028356206 0.0008082343 0.0019486236 0.005474476
3 1.407586e-07 1.143173e-04 0.0029373470 0.0015163033 0.0007975889 0.002338971
4 1.318936e-04 4.563723e-07 0.0062376362 0.0004532355 0.0064194526 0.001526317
5 1.950463e-03 1.678422e-03 0.0038656263 0.0040890780 0.0060264493 0.003238452
      increas          an       energi        more         but      provid
1 0.001677176 0.006458874 2.474225e-04 0.003380031 0.004795980 0.001251871
2 0.003653805 0.007163751 1.139505e-05 0.002497680 0.003429384 0.002108029
3 0.001243611 0.004401304 1.960397e-04 0.004070159 0.009042217 0.001568781
4 0.001154922 0.003544060 1.283958e-03 0.008630887 0.008423075 0.001040407
5 0.005893618 0.005633986 2.926456e-03 0.005131005 0.002396303 0.004073871
          law         need          had          at         gold       econom
1 0.003109904 7.512136e-05 0.0040114257 0.007714482 1.309228e-04 1.107365e-04
2 0.005475965 1.539361e-03 0.0009765216 0.007146743 1.424395e-03 3.853017e-05
3 0.003891993 3.424169e-03 0.0017757106 0.006929474 1.177787e-05 9.264097e-04
4 0.001257437 3.384718e-03 0.0012117336 0.004560909 2.421940e-05 1.243078e-03
5 0.001522363 4.843176e-03 0.0007360785 0.004073499 7.525389e-06 5.299745e-03
        there          ani         upon     countri          you       depart
1 0.001928146 0.0039241619 0.0047912778 0.004946326 1.880333e-03 0.0013806498
2 0.003608298 0.0029397285 0.0063484523 0.004510290 1.906905e-05 0.0042585130
3 0.005671389 0.0052464629 0.0033374460 0.003550335 1.286898e-03 0.0001714513
4 0.002570565 0.0011058898 0.0000860638 0.003220246 8.299901e-03 0.0001917486
5 0.001253666 0.0007819995 0.0005778589 0.002568331 3.786494e-09 0.0014159449
      american       these    constitut           so        world         also
1 0.0009485545 0.004027809 0.0033055139 0.0047983917 0.0005423394 0.0016484339
2 0.0034428782 0.004049241 0.0005961425 0.0009669202 0.0003542310 0.0009008162
3 0.0024349565 0.003751637 0.0008713312 0.

### tidy results

If you want to export these matrices as tidyverse-compatible tibbles—for example if you’re following the exercises in [Silge and Robinson](https://www.tidytextmining.com/index.html), you can use the helper verb `tidy()` and pass it the name of the matrix you want.

In [26]:
beta <- tidy(lda_model, matrix='beta')

head(beta)

topic,term,beta
<int>,<chr>,<dbl>
1,year,0.003793541
2,year,0.007826724
3,year,0.000787897
4,year,0.009357495
5,year,0.011326092
1,should,0.003293072


### Ranked component terms and topics

A more human-friendly interface is provided by topicmodels itself, in the form of the helper functions `terms()` and `topics()`. These provide, instead of the raw probabilities, a ranking of the top *n* terms per topic, `terms()`, or topics per document, `topics()`.

In [27]:
terms(lda_model, 20)

Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
state,govern,nation,year,year
their,state,should,american,program
from,year,or,more,nation
govern,an,all,but,congress
wa,at,but,you,feder
at,wa,thei,all,govern
unit,upon,their,peopl,new
all,from,at,their,these
an,should,can,can,increas
mai,such,must,america,an


In [28]:
topics(lda_model, 10)

199,158,119,201,122,110,113,117,67,111,⋯,14,2,21,22,128,1,11,7,12,169
5,5,3,5,2,2,3,3,1,2,⋯,1,1,1,1,3,1,1,1,1,5
4,3,2,4,3,1,2,2,3,1,⋯,3,4,2,5,2,3,2,3,3,3
2,2,5,1,1,3,1,5,2,3,⋯,4,3,3,2,1,4,3,4,5,4
1,1,1,2,5,5,5,1,5,5,⋯,5,2,5,3,5,5,4,5,4,1
3,4,4,3,4,4,4,4,4,4,⋯,2,5,4,4,4,2,5,2,2,2
